In [34]:
import pandas as pd
import json
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)

# pd.set_option('max_rows',20)
# pd.set_option('max_columns',10)
# pd.set_option('max_colwidth', 20)

In [35]:
df=pd.read_json('data_analytics_results.json').transpose()
#df.head()

### Data Cleaning

In [36]:
# cleaning contents
from bs4 import BeautifulSoup
def parse_html_to_text(content):
    return BeautifulSoup(content, features="html.parser").get_text()

df['contents'] = df['contents'].apply(parse_html_to_text)
df['contents'].iloc[0:4]

11414403                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [37]:
# Removing extra characters like '\t','\n' etc
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [38]:
df['publication_date'].str.split('T',expand=True)
df['date_published']=df['publication_date'].str.split('T',expand=True)[0]
df['time_published']=df['publication_date'].str.split('T',expand=True)[1]
df['time_published']=df['time_published'].str.replace('\D$','',regex=True)
df.drop('publication_date',axis=1,inplace=True)

In [39]:
#checking if all the values for df['type'] column is external
print(all(map(lambda x:x=='external',df['type'])))
print(df['model_type'].nunique())

False
1


In [40]:
df['type'].value_counts()

external    7904
native         1
Name: type, dtype: int64

In [41]:
df.set_index('id',inplace=True)
df.drop(['short_name','model_type','categories','type','refs','tags'],axis=1,inplace=True)

In [42]:
def clean_locations(loc):
   locations=[]
   #states=[]
   if loc!=None and len(loc)!=0:
      for i in range(len(loc)):
            locations.append(loc[i]['name'])
      #temp=loc[0]['name'].split(',')[1]
   return locations

In [43]:
df['locations']=df['locations'].apply(clean_locations)
df['levels']=df['levels'].apply(lambda level:level[0]['short_name'])
#df['tags']=df['tags'].apply(lambda tag: tag[0]['name'] if len(tag) != 0 else None)
#df['refs']=df['refs'].apply(lambda ref:ref['landing_page'])
df['company']=df['company'].apply(lambda company:company['name'])

In [44]:
def func_states(loc):
    if loc==None:
        return []
    elif 'Flexible / Remote' in loc:
        return ['Remote']
    else:
        lst=[]
        for i in range(len(loc)):
            if loc[i]!='United States':
                st=loc[i].split(',')[1].strip()
                if st not in lst:
                    lst.append(st)
        return lst



In [45]:
df['states']=df['locations'].apply(func_states)

In [46]:
# df_new=df[['name', 'states', 'levels', 'tags', 'company',
#        'date_published', 'time_published']].explode('states')
# df_new.reset_index()

In [48]:
df[['name', 'states', 'levels', 'company',
        'date_published', 'time_published']].to_csv('data_analytics_clean.csv')

In [49]:
df[['contents','locations']].to_csv('contents_locations')